In [483]:
import pandas as pd
import re

In [484]:
def clean_census(file, year):
    
    column_names= ['Variable', 'Arbutus-Ridge', 'Downtown', 'Dunbar-Southlands',
               'Fairview', 'Grandview-Woodland', 'Hastings-Sunrise',
               'Kensington-Cedar Cottage', 'Kerrisdale', 'Killarney', 'Kitsilano',
               'Marpole', 'Mount Pleasant', 'Oakridge', 'Renfrew-Collingwood',
               'Riley Park', 'Shaughnessy', 'South Cambie', 'Strathcona',
               'Sunset', 'Victoria-Fraserview', 'West End', 'West Point Grey',
               'Vancouver CSD', 'Vancouver CMA']
   
    # read in csv file as dataframe
    df = pd.read_csv(file, encoding='latin-1', skiprows=4)
    
    # remove 'ID' column if present
    df.drop(columns='ID', inplace=True, errors='ignore')
    
    # rename columns 
    df.set_axis(column_names, axis=1, inplace=True)
    
    # remove empty rows
    df.dropna(0, 'all', inplace=True)
    
    # remove leading whitespace from variables
    df.Variable = df.Variable.apply(lambda x: x.lstrip())
    df.drop(df[df.Variable.str.contains('20%.*data', flags=re.IGNORECASE) ].index , inplace=True)
    
    # convert all data to strings except NaN values
    df.iloc[:,1:25] = df.iloc[:,1:25].applymap(lambda x: str(x) if x == x else x)
    
    # convert data to float
    df.iloc[:,1:25] = df.iloc[:,1:25].applymap(lambda x: re.sub("[-]", "0", x) if x == x else x)
    df.iloc[:,1:25] = df.iloc[:,1:25].applymap(lambda x: float(re.sub("[,$]", "", x)) if x == x else x)
    
    # divide the census datasets into subgroups
    sub_dict = create_subgroup_dict(df, year)
    
    # clean the age datatables
    census_dict = clean_age(sub_dict, year)
    
    census_dict = clean_marital_status(census_dict, year)
    
    census_dict = clean_private_households(census_dict, year)
    census_dict = clean_fam_structure(census_dict, year)
        
    return census_dict

In [508]:
def create_subgroup_dict(df, year):
    
    # separate dataframe by 'Variables' containing regex expressions:
    if year == 2001:
        re1 = ['total.*by', 
               'population.*by', 
               'common-law couples',
               '^Male', 
               '^Female', 
               'total - male', 
               'total - female']
        
    elif year == 2006:
        re1 = ['total.*by', 
               'population.*by',
               'common-law couples',
               '^Male[s\s,]', 
               '^Female[s\s,]']
        
    elif year == 2011:
        df.drop(index=201 , inplace=True)
        re1 = ['total.*by', 
               'population.*by', 
               'common-law couples',
               'males',
               'Total population excluding institutional residents',  
               'Total.*in private households'] 
    else:
        re1 = ['^total', 'population.*by', 'males']
    
    subgroup = list(df[df.Variable.str.contains('|'.join(re1), flags=re.IGNORECASE)].index)
    subgroup = subgroup[1:]
    
    # create census dictionary of sub datasets
    # initialize variables for the lookup dictionary
    start = 0
    census_dict = {}
    
    for s in subgroup:
        sub_df = df.loc[start:s-1]

        # transpose dataframe and rename column
        sub_df = sub_df.set_index('Variable').T.reset_index().rename(columns={'index': 'LocalArea'})

        # clean up names and store dataframes into the dictionary
        census_dict[df.Variable[start].rstrip().lstrip()] = sub_df
        start = s
        
    return census_dict


In [509]:
def clean_age(census_dict, year):
    
    if year == 2001:
        column_names = ['LocalArea', 'Type', 'Total', '0 to 4 years', 
                   '5 to 9 years', '10 to 14 years', '15 to 19 years',
                   '20 to 24 years', '25 to 29 years', '30 to 34 years', 
                   '35 to 39 years', '40 to 44 years', '45 to 49 years', 
                   '50 to 54 years', '55 to 59 years', '60 to 64 years', 
                   '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                   '80 to 84 years', '85 to 89 years', '90 to 94 years',
                   '95 to 99 years', '100 years and over']

        male = census_dict['Male']
        female = census_dict['Female']
    
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male])
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        total = merged.groupby('LocalArea').sum()
        total['Type'] = 'total'
        total.reset_index(inplace=True)
        merged = pd.concat([merged, total])
    
    else:
        if year == 2006:
            
            column_names = ['LocalArea', 'Type', 'Total', '0 to 4 years', 
                       '5 to 9 years', '10 to 14 years', '15 to 19 years',
                       '20 to 24 years', '25 to 29 years', '30 to 34 years', 
                       '35 to 39 years', '40 to 44 years', '45 to 49 years', 
                       '50 to 54 years', '55 to 59 years', '60 to 64 years', 
                        '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                       '80 to 84 years', '85 to 89 years', '90 to 94 years',
                       '95 to 99 years', '100 years and over', 'Median Age']
            
            total = census_dict['Male & Female, Total']
            male = census_dict['Male, Total']
            female = census_dict['Female, Total']
        
        elif year == 2011:
            
            column_names = ['LocalArea', 'Type', 'Total', '0 to 4 years', 
                       '5 to 9 years', '10 to 14 years', '15 to 19 years',
                       '15 years', '16 years', '17 years', '18 years', '19 years',
                       '20 to 24 years', '25 to 29 years', '30 to 34 years', 
                       '35 to 39 years', '40 to 44 years', '45 to 49 years', 
                       '50 to 54 years', '55 to 59 years', '60 to 64 years', 
                       '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                       '80 to 84 years', '85 years and over', 'Median age',
                       '% of the population aged 15 and over']
        
            total = census_dict['Total population by age groups']
            male = census_dict['Males, total']
            female = census_dict['Females, total']

        elif year == 2016:
            
            column_names = ['LocalArea', 'Type', 'Total', '0 to 14 years', 
                       '0 to 4 years', '5 to 9 years', '10 to 14 years',
                       '15 to 64 years', '15 to 19 years', '20 to 24 years', 
                       '25 to 29 years', '30 to 34 years', '35 to 39 years', 
                       '40 to 44 years', '45 to 49 years', '50 to 54 years', 
                       '55 to 59 years', '60 to 64 years', '65 years and over', 
                       '65 to 69 years', '70 to 74 years', '75 to 79 years', 
                       '80 to 84 years', '85 years and over','85 to 89 years', 
                       '90 to 94 years', '95 to 99 years', '100 years and over']
        
            total = census_dict['Total - Age groups and average age of the population - 100% data']
            male = census_dict['Total - Age groups and average age of males - 100% data']
            female = census_dict['Total - Age groups and average age of females - 100% data']
         
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
        total.insert(1, 'Type', 'total')
        total.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male, total])
    
    merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
    census_dict['population by age and sex'] = merged
    
    return census_dict

In [510]:
def clean_marital_status(census_dict, year):
    
    if year in [2001, 2006]:
        column_names = ['LocalArea', 'Total population 15 years and over',
                        'Single (never legally married)',
                       'Married', 'Separated', 'Divorced',
                       'Widowed', 'total x', 'Not living common law',
                       'Living common law']
        
        columns_ordered = ['LocalArea', 'Total population 15 years and over',
                       'Married or living with a or common-law partner',
                       'Married', 'Living common law',
                       'Not living with a married spouse or common-law partner',
                       'Single (never legally married)', 'Separated', 'Divorced',
                       'Widowed']
        
        df1 = census_dict['Total population 15 years and over by legal marital status']
        df2 = census_dict['Total population 15 years and over by common-law status']
        
        merged = pd.merge(df1, df2, on=['LocalArea'])
        merged.set_axis(column_names, axis=1, inplace=True)

        merged['Married or living with a or common-law partner'] = merged['Married'] + merged['Living common law']
        merged['Not living with a married spouse or common-law partner'] = merged['Total population 15 years and over'] - merged['Married or living with a or common-law partner']
        merged = merged[columns_ordered]
        census_dict['marital status'] = merged
    
    else:
        if year == 2011:
            total = census_dict['Total population 15 years and over by marital status']
            male = census_dict['Males 15 years and over by marital status']
            female = census_dict['Females 15 years and over by marital status']
        elif year == 2016:
            total = census_dict['Total - Marital status for the population aged 15 years and over - 100% data']
            male = census_dict['Total - Marital status for males aged 15 years and over - 100% data']
            female = census_dict['Total - Marital status for females aged 15 years and over - 100% data']
            
        column_names = ['LocalArea', 'Type', 'Total population 15 years and over',
                       'Married or living with a or common-law partner',
                       'Married', 'Living common law',
                       'Not living with a married spouse or common-law partner',
                       'Single (never legally married)', 'Separated', 'Divorced',
                       'Widowed']
        
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
        total.insert(1, 'Type', 'total')
        total.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male, total])
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        census_dict['marital status'] = merged
        
        
    return census_dict

In [511]:
def clean_private_households(census_dict, year):
    if year == 2011:
        column_names = ['LocalArea', 'Type', 'Persons in private households ',
                   'Persons not in census families', 'Living with relatives',
                   'Living with non-relatives only', 'Living alone',
                   'Number of census family persons']
        
        total = census_dict['Total number of persons in private households']
        male = census_dict['Number of males in private households']
        female = census_dict['Number of females in private households']
        total_65 = census_dict['Total number of persons aged 65 years and over in private households']
        male_65 = census_dict['Number of males aged 65 years and over in private households']
        female_65 = census_dict['Number of females aged 65 years and over in private households']
        
        female.insert(1, 'Type', 'female')
        female.set_axis(column_names, axis=1, inplace=True)
        male.insert(1, 'Type', 'male')
        male.set_axis(column_names, axis=1, inplace=True)
        total.insert(1, 'Type', 'total')
        total.set_axis(column_names, axis=1, inplace=True)
        
        female_65.insert(1, 'Type', '65+ female')
        female_65.set_axis(column_names, axis=1, inplace=True)
        male_65.insert(1, 'Type', '65+ male')
        male_65.set_axis(column_names, axis=1, inplace=True)
        total_65.insert(1, 'Type', '65+ total')
        total_65.set_axis(column_names, axis=1, inplace=True)
    
        merged = pd.concat([female, male, total, female_65, male_65, total_65])
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        census_dict['private households - individuals'] = merged

    return census_dict

In [512]:
def clean_fam_structure(census_dict, year):
    
    column_names = ['LocalArea', 'Type', 'Total', 
                    'Without children at home',
                    'With children at home', '1 child', '2 children',
                    '3 or more children']

    if year == 2016:
        total = census_dict['Total - Couple census families in private households - 100% data']
        total.insert(1, 'Type', 'total couples')
        total.set_axis(column_names, axis=1, inplace=True)
    
        census_dict['family structure'] = total
    else:
        if year in [2011, 2006]:
            married = census_dict['Total couple families by family structure and number of children'] 
            married = married[['LocalArea', 'Married couples ',
                               'Without children at home ',
                               'With children at home ', '1 child ', '2 children ',
                               '3 or more children ']]
            common_law = census_dict['Common-law couples']
            
        elif year == 2001:
            married = census_dict['Total couple families by family structure']
            married = married[['LocalArea','Married couples', 
                               'Without children at home', 'With children at home',
                               '1 child', '2 children', '3 or more children']]
            common_law = census_dict['Common-law couples']
            
        married.insert(1, 'Type', 'married couples')
        married.set_axis(column_names, axis=1, inplace=True)
        common_law.insert(1, 'Type', 'common-law couples')
        common_law.set_axis(column_names, axis=1, inplace=True)
        
        merged = pd.concat([married, common_law])
        total = merged.groupby('LocalArea').sum()
        total['Type'] = 'total couples'
        total.reset_index(inplace=True)
        merged = pd.concat([merged, total])
        
        merged.sort_values(by=['LocalArea', 'Type'], inplace=True)
        census_dict['family structure'] = merged
    
    return census_dict
        
        

In [513]:
df_2001 = clean_census('../data/raw/census_2001.csv', 2001)
df_2006 = clean_census('../data/raw/census_2006.csv', 2006)
df_2011 = clean_census('../data/raw/census_2011.csv', 2011)
df_2016 = clean_census('../data/raw/census_2016.csv', 2016)

In [514]:
for key in df_2006.keys():
    print(key)

Total population by sex and age groups - 100% data
Male & Female, Total
Male, Total
Female, Total
Total population 15 years and over by legal marital status
Total population 15 years and over by common-law status
Total population by mother tongue
Total population by knowledge of official languages
Total population by first official language spoken
Total population by language spoken most often at home
Total population by citizenship
Total population by immigrant status and place of birth
Total recent immigrants by selected places of birth
Total immigrant population by period of immigration
Total immigrant population by age at immigration
Total population 15 years and older by generation status
Total population by Aboriginal and non-Aboriginal identity population
Total population by Registered Indian status
Total population 15 years and over by labour force activity
Males 15 years and over - Labour force activity
Males 15 to 24 years - Labour force activity
Males 25 years and over - Lab

In [515]:
df_2006['family structure']


,LocalArea,Type,Total,Without children at home,With children at home,1 child,2 children,3 or more children
0,Arbutus-Ridge,common-law couples,200,145,55,35,20,10
0,Arbutus-Ridge,married couples,3390,1140,2250,840,1040,370
0,Arbutus-Ridge,total couples,3590,1285,2305,875,1060,380
1,Downtown,common-law couples,2390,2220,170,75,20,70
1,Downtown,married couples,6160,3510,2645,1605,895,145
...,...,...,...,...,...,...,...,...
20,West End,married couples,4790,3230,1560,1040,465,60
22,West End,total couples,7830,6025,1805,1205,545,60
21,West Point Grey,common-law couples,395,275,120,80,40,0
21,West Point Grey,married couples,2650,1030,1625,515,875,235


In [454]:
df_2011['family structure']

,LocalArea,Type,Total,Without children at home,With children at home,1 child,2 children,3 or more children
0,Arbutus-Ridge,common-law couples,235,185,55,25,30,5
0,Arbutus-Ridge,married couples,3270,1060,2205,905,1010,300
0,Arbutus-Ridge,total couples,3505,1245,2260,930,1040,305
1,Downtown,common-law couples,3380,3145,235,180,50,10
1,Downtown,married couples,7700,4810,2890,1775,955,155
...,...,...,...,...,...,...,...,...
20,West End,married couples,4555,3280,1275,840,385,45
22,West End,total couples,7840,6385,1455,975,420,50
21,West Point Grey,common-law couples,380,280,100,50,30,10
21,West Point Grey,married couples,2670,1090,1580,565,770,245


In [442]:
df_2011['Total children in census families in private households']


Variable,LocalArea,Total children in census families in private households,Under six years of age,6 to 14 years,15 to 17 years,18 to 24 years,25 years and over,Average number of children at home per census family,Average number of persons per census family
0,Arbutus-Ridge,5110,695,1615,840,1245,720,1.2,3
1,Downtown,6320,2075,1295,440,1305,1200,0.5,2.4
2,Dunbar-Southlands,7440,1190,2620,935,1750,945,1.2,3.1
3,Fairview,3640,1215,970,285,655,505,0.5,2.4
4,Grandview-Woodland,5195,1385,1520,540,885,865,0.8,2.6
5,Hastings-Sunrise,10625,1930,3030,1120,2190,2355,1.1,3
6,Kensington-Cedar Cottage,14615,2880,4075,1565,2960,3140,1.2,3
7,Kerrisdale,4780,635,1425,680,1350,700,1.2,3
8,Killarney,9515,1605,2575,1110,2200,2020,1.2,3
9,Kitsilano,6465,1975,1925,660,1210,690,0.7,2.5
